## load Model
---

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.models import Model as Kerasmodel

%load_ext autoreload
%autoreload 2

original_model = tf.keras.models.load_model("/home/jovyan/models/origin/BirdNET-Analyzer/checkpoints/V2.1/BirdNET_GLOBAL_2K_V2.1_Model/")

2023-08-31 08:47:27.109530: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-31 08:47:27.152690: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-31 08:47:27.153492: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 08:47:27.938155: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Remove last layer and save
---

In [2]:
no_class_model = Kerasmodel(original_model.input, original_model.layers[-2].output)
no_class_model_path = "/home/jovyan/models/checkpoints_/no_class_model.tflite"

# Convert the keras model to a tflite
converter = tf.lite.TFLiteConverter.from_keras_model(no_class_model)
tflite_no_class_model = converter.convert()

print("Finished Converting")

with open(no_class_model_path, 'wb') as f:
    f.write(tflite_no_class_model)
print("Finished Saving")

INFO:tensorflow:Assets written to: /tmp/tmp1724hl03/assets


INFO:tensorflow:Assets written to: /tmp/tmp1724hl03/assets
2023-08-31 08:48:30.219405: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-08-31 08:48:30.219469: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-08-31 08:48:30.220277: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp1724hl03
2023-08-31 08:48:30.252069: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-08-31 08:48:30.252126: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmp1724hl03
2023-08-31 08:48:30.313639: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2023-08-31 08:48:30.339092: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-08-31 08:48:30.739769: I tensorflow/cc/saved_model/loader.cc:215] Running initializatio

Finished Converting
Finished Saving


## Load training data
---

In [3]:
import data as data


train_data_path = "/home/jovyan/cut-data/training/"

# Load training data
print('Loading training data...', flush=True)
x_train, y_train, labels, file_paths_train = data.loadData(train_data_path)
print('...Done. Loaded {} training samples and {} labels.'.format(x_train.shape[0], y_train.shape[1]), flush=True)

Loading training data...
Processed 0 files. Currently processing file: /home/jovyan/cut-data/training/non_target/0.920_0001_127764.wav
Processed 100 files. Currently processing file: /home/jovyan/cut-data/training/non_target/0.920_0003_82178411.wav
Processed 200 files. Currently processing file: /home/jovyan/cut-data/training/non_target/0.920_0006_718263.wav
Processed 300 files. Currently processing file: /home/jovyan/cut-data/training/non_target/0.920_0010_540556.wav
Processed 400 files. Currently processing file: /home/jovyan/cut-data/training/non_target/0.920_0016_324707.wav
Processed 500 files. Currently processing file: /home/jovyan/cut-data/training/non_target/0.920_0022_355153.wav
Processed 600 files. Currently processing file: /home/jovyan/cut-data/training/non_target/0.920_0031_270434051.wav
Processed 700 files. Currently processing file: /home/jovyan/cut-data/training/non_target/0.920_0043_647427.wav
Processed 800 files. Currently processing file: /home/jovyan/cut-data/traini

In [6]:
import numpy as np

# balance the training data:
print('Balance the training data...')

y_train_indices = np.argmax(y_train, axis=1)

# minimum of one class
min_samples = min(np.bincount(y_train_indices))

# reduce entries until minimum after shuffle 
balanced_x_train = []
balanced_y_train = []
balanced_file_paths_train = []
for label in np.unique(y_train_indices):
    indices = np.where(y_train_indices == label)[0]
    np.random.shuffle(indices)  # Random order for random removal of samples
    indices = indices[:min_samples]
    balanced_x_train.append(x_train[indices])
    balanced_y_train.append(y_train[indices])
    balanced_file_paths_train.extend(file_paths_train[indices])

# Combine the balanced data for all classes
balanced_x_train = np.concatenate(balanced_x_train, axis=0)
balanced_y_train = np.concatenate(balanced_y_train, axis=0)
balanced_file_paths_train = np.array(balanced_file_paths_train)

print('Balanced training data:')
print('x_train shape:', balanced_x_train.shape)
print('y_train shape:', balanced_y_train.shape)
print('file_paths_train shape:', balanced_file_paths_train.shape)

print('...Done. Loaded {} training samples and {} labels.'.format(balanced_x_train.shape[0], balanced_y_train.shape[0]), flush=True)

Balance the training data...
Balanced training data:
x_train shape: (7612,)
y_train shape: (7612, 2)
file_paths_train shape: (7612,)
...Done. Loaded 7612 training samples and 7612 labels.


## Load validation data and convert
---

In [7]:
val_data_path = "/home/jovyan/cut-data/validation/"

print('Loading validation data...', flush=True)
x_val, y_val, labels, file_paths_val = data.loadData(val_data_path)
print('...Done. Loaded {} validation samples and {} labels.'.format(x_val.shape[0], y_val.shape[1]), flush=True)

Loading validation data...
Processed 0 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/0.920_0001_270097.wav
Processed 100 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/0.930_0002_182583971.wav
Processed 200 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/0.940_0004_534761.wav
Processed 300 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/0.950_0018_226391901.wav
Processed 400 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/0.970_0017_647758.wav
Processed 500 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/1.000_0001_0H2uMhzSitY_520.wav
Processed 600 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/1.000_0002_--ivFZu-hlc_30.wav
Processed 700 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/1.000_0002_2RpOd9MJjyQ_10.wav
Processed 800 files. Currently p

In [8]:
# Convert to numpy arrays

import numpy as np

x_train = np.array(x_train, dtype='float32')
y_train = np.array(y_train, dtype='float32')

x_val = np.array(x_val, dtype='float32')
y_val = np.array(y_val, dtype='float32')

print('...Done. Loaded {} training files and {} labels.'.format(x_train.shape[0], y_train.shape[1]), flush=True)
print('...Done. Loaded {} validation files and {} labels.'.format(x_val.shape[0], y_val.shape[1]), flush=True)

...Done. Loaded 11292 training files and 2 labels.
...Done. Loaded 1381 validation files and 2 labels.


## Create new 2 Class Model
---

## Build custom classifier
---

In [9]:
import model

train_hidden_units = 0

# Build two class model
print('Building model...', flush=True)    
two_class_model_softmax = model.addLinearClassifier(no_class_model,
                                            y_train.shape[1], #number of labels
                                            train_hidden_units) #number of hidden units

print('...Done.', flush=True)

Building model...
...Done.


### check new shape

In [10]:
two_class_model_softmax.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 INPUT (InputLayer)          [(None, 144000)]             0         []                            
                                                                                                  
 ADVANCED_SPEC1 (LinearSpec  (None, 128, 513, 1)          1         ['INPUT[0][0]']               
 Layer)                                                                                           
                                                                                                  
 BNORM_SPEC_NOQUANT (BatchN  (None, 128, 513, 1)          4         ['ADVANCED_SPEC1[0][0]']      
 ormalization)                                                                                    
                                                                                            

# Train Two Class Model
---

## Train Model
---

### NaN Debug (not active)

#### Trainingsdata

In [8]:
# import model
# # Disable Model Debug
# tf.debugging.disable_check_numerics()

# nan_samples = model.debug_model(two_class_model, file_paths_train, y_train, train_batch_size)

#### Validationdata

In [9]:
# import model
# # Disable Model Debug
# tf.debugging.disable_check_numerics()

# nan_samples = model.debug_model(two_class_model, file_paths_val, y_val, train_batch_size)

### Actual Training

In [11]:
train_epochs = 5
train_batch_size = 12
train_learning_rate = 0.0001
    
# Number of last layers to be trained, rest will be frozen, 0 = alle will be trained
train_layers_num = 2

on_epoch_end = None

print("Shape of training labels: ", x_train.shape)
print("Shape of validation labels: ", x_val.shape)

Shape of training labels:  (11292,)
Shape of validation labels:  (1381,)


In [12]:
# Train model
print('Training model...', flush=True)

# Check for Errors to find NaN problem
# tf.debugging.enable_check_numerics()

# Disable Model Debug
tf.debugging.disable_check_numerics()

trained_two_class_model_softmax, history = model.trainNewModel(
                                      two_class_model_softmax,
                                      train_layers_num,
                                      balanced_x_train, 
                                      balanced_y_train,
                                      x_val,
                                      y_val,
                                      balanced_file_paths_train,
                                      file_paths_val,
                                      epochs=train_epochs,
                                      batch_size=train_batch_size,
                                      learning_rate=train_learning_rate,
                                      on_epoch_end=on_epoch_end)

print('Done', flush=True)

Training model...
Length of trainable Weightslen  2
Epoch 1/5
635/635 [==============================] - 432s 676ms/step - loss: 0.4381 - accuracy: 0.8988 - prec: 0.8988 - recall: 0.8988 - val_loss: 0.2439 - val_accuracy: 0.9986 - val_prec: 0.9986 - val_recall: 0.9986
Epoch 2/5
635/635 [==============================] - 103s 162ms/step - loss: 0.1704 - accuracy: 0.9995 - prec: 0.9995 - recall: 0.9995 - val_loss: 0.1264 - val_accuracy: 1.0000 - val_prec: 1.0000 - val_recall: 1.0000
Epoch 3/5
635/635 [==============================] - 101s 160ms/step - loss: 0.1049 - accuracy: 0.9999 - prec: 0.9999 - recall: 0.9999 - val_loss: 0.0917 - val_accuracy: 1.0000 - val_prec: 1.0000 - val_recall: 1.0000
Epoch 4/5
635/635 [==============================] - 103s 162ms/step - loss: 0.0837 - accuracy: 1.0000 - prec: 1.0000 - recall: 1.0000 - val_loss: 0.0791 - val_accuracy: 1.0000 - val_prec: 1.0000 - val_recall: 1.0000
Epoch 5/5
635/635 [==============================] - 106s 167ms/step - loss: 0.0

### TESTING

In [13]:
import evaluateModel

test_data_path = "/home/jovyan/cut-data/testing/"

evaluateModel.evaluateModel(trained_two_class_model_softmax, test_data_path, train_batch_size)


Processed 0 files. Currently processing file: /home/jovyan/cut-data/testing/non_target/0.920_0001_703520.wav
Processed 100 files. Currently processing file: /home/jovyan/cut-data/testing/non_target/0.920_0057_645986.wav
Processed 200 files. Currently processing file: /home/jovyan/cut-data/testing/non_target/0.930_0263_741788.wav
Processed 300 files. Currently processing file: /home/jovyan/cut-data/testing/non_target/0.950_0002_645965.wav
Processed 400 files. Currently processing file: /home/jovyan/cut-data/testing/non_target/0.960_0024_103739801.wav
Processed 500 files. Currently processing file: /home/jovyan/cut-data/testing/non_target/1.000_0001_-X5Ay0Wuew0_20.wav
Processed 600 files. Currently processing file: /home/jovyan/cut-data/testing/non_target/1.000_0001_4TQzd0lB8IQ_30.wav
Processed 700 files. Currently processing file: /home/jovyan/cut-data/testing/non_target/1.000_0002_1MF9_29YUZU_10.wav
Processed 800 files. Currently processing file: /home/jovyan/cut-data/testing/non_targe

### SAVE
---

#### Save Baseline Model with last Activation Layer

In [15]:
tflite_baselineModel_activation_path = "/home/jovyan/models/checkpoints_/baseline_two_class_model_softmax_activation.tflite"
keras_baselineModel_activation_path = "/home/jovyan/models/checkpoints_/baseline_two_class_model_softmax_activation/"

baselineModel_activation_softmax = trained_two_class_model_softmax

baselineModel_activation_softmax.summary()

# Save the model to a keras model
baselineModel_activation_softmax.save(keras_baselineModel_activation_path)
print("Finished Keras File save")

# Convert the keras model to a tflite
converter = tf.lite.TFLiteConverter.from_keras_model(baselineModel_activation_softmax)
tflite_baselineModel_activation_softmax = converter.convert()

print("Finished Converting")

with open(tflite_baselineModel_activation_path, 'wb') as f:
    f.write(tflite_baselineModel_activation_softmax)
print("Finished Saving")

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 INPUT (InputLayer)          [(None, 144000)]             0         []                            
                                                                                                  
 ADVANCED_SPEC1 (LinearSpec  (None, 128, 513, 1)          1         ['INPUT[0][0]']               
 Layer)                                                                                           
                                                                                                  
 BNORM_SPEC_NOQUANT (BatchN  (None, 128, 513, 1)          4         ['ADVANCED_SPEC1[0][0]']      
 ormalization)                                                                                    
                                                                                            

INFO:tensorflow:Assets written to: /home/jovyan/models/checkpoints_/baseline_two_class_model_softmax_activation/assets


Finished Keras File save
INFO:tensorflow:Assets written to: /tmp/tmphehizlz4/assets


INFO:tensorflow:Assets written to: /tmp/tmphehizlz4/assets
2023-08-31 09:34:58.283324: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-08-31 09:34:58.283390: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-08-31 09:34:58.283704: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmphehizlz4
2023-08-31 09:34:58.328820: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-08-31 09:34:58.328877: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmphehizlz4
2023-08-31 09:34:58.434187: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-08-31 09:34:58.881912: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /tmp/tmphehizlz4
2023-08-31 09:34:59.087488: I tensorflow/cc/saved_model/loader.cc:314] SavedModel

Finished Converting
Finished Saving


#### Eval saved tfLite Model

In [ ]:
import evaluateTFliteModel

tflite_baselineModel_activation_path = "/home/jovyan/models/checkpoints_/baseline_two_class_model_activation.tflite"
test_data_path = "/home/jovyan/cut-data/testing/"

batch_size = 12

evaluateTFliteModel.evaluate_tflite_model(tflite_baselineModel_activation_path, test_data_path, batch_size)

#### Save Baseline Model without last Activation Layer for inference using Sensitivity (not active)

In [ ]:
# tflite_baselineModel_path = "/home/jovyan/models/checkpoints_/baseline_two_class_model_activationremoved.tflite"

# # Remove activation layer, logistic to have sensitivity option later with sigmoid
# baselineModel = Kerasmodel(trained_two_class_model.input, trained_two_class_model.layers[-2].output)

# baselineModel.summary()

# # Convert the keras model to a tflite
# converter = tf.lite.TFLiteConverter.from_keras_model(baselineModel)
# tflite_baselineModel = converter.convert()

# print("Finished Converting")

# with open(tflite_baselineModel_path, 'wb') as f:
#     f.write(tflite_baselineModel)
# print("Finished Saving")